In [1]:
from tx import TxTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(TxTest())
unittest.TextTestRunner().run(suite)


..............
----------------------------------------------------------------------
Ran 14 tests in 0.801s

OK


<unittest.runner.TextTestResult run=14 errors=0 failures=0>

In [2]:
from helper import HelperTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(HelperTest())
unittest.TextTestRunner().run(suite)

......
----------------------------------------------------------------------
Ran 6 tests in 0.007s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

In [3]:
from binascii import hexlify, unhexlify
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

tx_ins = []
prev_tx = unhexlify('0025bc3c0fa8b7eb55b9437fdbd016870d18e0df0ace7bc9864efc38414147c8')
tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=0,
            script_sig=b'',
            sequence=0xffffffff,
        ))
tx_outs = []
h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
tx_outs.append(TxOut(
    amount=int(0.99*100000000),
    script_pubkey=p2pkh_script(h160),
))
h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
tx_outs.append(TxOut(
    amount=int(0.1*100000000),
    script_pubkey=p2pkh_script(h160),
))
tx = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)
sighash = SIGHASH_ALL
z = tx.hash_to_sign(0, sighash)
pk = PrivateKey(secret=8675309)
der = pk.sign(z).der()
sig = der + bytes([sighash])
sec = unhexlify(pk.point.sec())
script_sig = bytes([len(sig)]) + sig + bytes([len(sec)]) + sec
script_sig = bytes([len(script_sig)]) + script_sig
tx.tx_ins[0].script_sig = Script.parse(script_sig)
print(hexlify(tx.serialize()))

b'0100000001c847414138fc4e86c97bce0adfe0180d8716d0db7f43b955ebb7a80f3cbc2500000000006c6b483045022100d40207a9751cbdebf1ac69ffc253641ab50ffb1e420bd376ab0ff9641f315135022022f96ffc14b1b854520de22f1bb1891450d1cd2c0f417321e0c55fa4ad0e3f52012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff02c09ee605000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac80969800000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88ac00000000'


In [4]:
# Exercise 3

from binascii import hexlify, unhexlify
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from tx import TxIn, TxOut, Tx

private_key = PrivateKey(8675309)
tx_ins = []
prev_tx = unhexlify('7eeb14c48a95f517e9c2cf22214da7d1759caa02211e3a76de5336c5fc339f86')
tx_ins.append(TxIn(
    prev_tx=prev_tx,
    prev_index=0,
    script_sig=b'',
    sequence=0xffffffff,
))
tx_outs = []
h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
tx_outs.append(TxOut(
    amount=int(0.05*100000000),
    script_pubkey=p2pkh_script(h160),
))
h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
tx_outs.append(TxOut(
    amount=int(0.049*100000000),
    script_pubkey=p2pkh_script(h160),
))
tx = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)
tx.sign_input(0, private_key, SIGHASH_ALL)
print(hexlify(tx.serialize()))

b'0100000001869f33fcc53653de763a1e2102aa9c75d1a74d2122cfc2e917f5958ac414eb7e000000006b483045022100b24d980f243ee0a50c5c9be2de51513ec5c92aa7af08a5d1280fd845cb12a37202207c2a81895dd3d02d7f33dc2f5fa4d3cf4b158783c175402133007536df0f7d71012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff02404b4c00000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88aca0c44a00000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac00000000'


In [5]:
# Exercise 4

from binascii import hexlify, unhexlify
from helper import hash160

h160 = hash160(unhexlify('5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'))
print(hexlify(h160))

b'74d691da1574e6b3c192ecfb52cc8984ee7b6c56'


In [6]:
from binascii import unhexlify
from helper import encode_base58_checksum

print(encode_base58_checksum(b'\x6f'+unhexlify('74d691da1574e6b3c192ecfb52cc8984ee7b6c56')))

mrAjisaT4LXL5MzE81sfcDYKU3wqWSvf9q


In [7]:
from binascii import unhexlify
from helper import double_sha256

sha = double_sha256(unhexlify('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
                    
print(hex(z))

0xe71bfa115715d6fd33796948126f40a8cdd39f187e4afb03896795189fe1423c


In [8]:
from binascii import hexlify, unhexlify
from ecc import S256Point, Signature
from helper import double_sha256

sha = double_sha256(unhexlify('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
point = S256Point.parse(unhexlify('022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb70'))
r = 0x00dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e
s = 0x5a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a89937
sig = Signature(r,s)
print(point.verify(z, sig))

True


In [9]:
# Exercise 6.1

from binascii import hexlify, unhexlify
from ecc import S256Point, Signature
from helper import double_sha256

sha = double_sha256(unhexlify('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
point = S256Point.parse(unhexlify('03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71'))
r = 0x00da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b
s = 0x7bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e754022
sig = Signature(r,s)
print(point.verify(z, sig))

True
